In [ ]:
from ipywidgets import *
from IPython.display import IFrame
from IPython.core.display import display
import subprocess, sys
import webbrowser
from ipywidgets import HTML
import base64
import csv ## https://docs.python.org/fr/3.6/library/csv.html 
import os ## Pour utiliser la methode ci-dessous Get Current Worked Directory pour lire la page Web crée dans le dossier courant.
Lieu = os.getcwd()  ## methode Get Current Worked Directory pour lire la page Web crée dans le dossier courant.
###
import os

import subprocess, sys

import time
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
#################################################
def Recup_Donnees(titre_csv):
    """ Cette fonction initiale et essentielle permet de construire
        une liste de tuples contenant toutes les informations du 
        tableau par lignes. """
    Donnees=[]  ## liste de tuples pour récupérer toutes les données du fichier csv
    Ligne_Donnees=[]  ## sous liste pour créer la liste des tuples de Donnees
    with open(titre_csv, "r", encoding="utf-8") as texte:  ## Ouverture du csv en mode utf 8 pour encodage des caractères français
        Les_lignes = csv.reader(texte, delimiter=';') ## lecture de chaque ligne du fichier csv
        for ligne in Les_lignes:  ## bouclage sur chaque ligne des lignes
            for element in ligne:  ## Récupération de chaque élément de chaque ligne
                Ligne_Donnees.append(element)  ## création d'1 tuple avec tous les éléments d'1 même ligne
            Ligne_Donnees=tuple(Ligne_Donnees)  ## assurance qu'il s'agit bien d'un tuple ..?
            Donnees.append(Ligne_Donnees)  ## Ajout du tuple créé à la liste de tuple Données
            Ligne_Donnees=[]  ## Réinitialisation d'1 futur 'tuple' vide pour boucler sur la ligne suivant du tableau
    longueur_Donnees=len(Donnees)  ## Détermination de la longueurs des Données
    return Donnees,longueur_Donnees
Donnees,longueur_Donnees=Recup_Donnees("Entreprises_Complet.csv")
# print("longueur = ",longueur_Donnees)

In [ ]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster,LayersControl,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    Popup,
    GeoJSON,
    DrawControl,
    basemaps, FullScreenControl,
    SearchControl, AwesomeIcon, Icon
)

from ipywidgets import HTML

In [ ]:
def html_button_fonc(_):
    global selection
    res = str(selection)
#     res = '''salut david,
#     comment ça va ?
#     moi ça va super,

#     et toi ?'''



    #FILE
    filename = 'res.txt'
    b64 = base64.b64encode(res.encode())
    payload = b64.decode()
    print(type(payload))

    #BUTTONS
    html_buttons = '''<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="{filename}" href="data:text/csv;base64,{payload}" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Ici</button>
    </a>
    </body>
    </html>
    '''

    html_button = html_buttons.format(payload=payload,filename=filename)
    display(HTML(html_button))


In [ ]:
def drop_fonc(change):
    global memoir_domain # m, marker_cluster
    domain=change['new']
    
    if memoir_domain == "Tous":
        if domain == "Pharmaceutique":
            m.remove_layer(cluster_cosmeto)
            m.remove_layer(cluster_alim)
            memoir_domain = "Pharma"
            return memoir_domain
        if domain == "Cosmétique":
            m.remove_layer(cluster_pharma)
            m.remove_layer(cluster_alim)
            memoir_domain = "Cosmeto"
            return memoir_domain
        if domain == "Alimentaire":
            m.remove_layer(cluster_pharma)
            m.remove_layer(cluster_cosmeto)
            memoir_domain = "Alim"
            return memoir_domain
        
    if memoir_domain == "Pharma":
        if domain == "Tous":
            m.add_layer(cluster_cosmeto)
            m.add_layer(cluster_alim)
            memoir_domain = "Tous"
            return memoir_domain
        if domain == "Cosmétique":
            m.remove_layer(cluster_pharma)
            m.add_layer(cluster_cosmeto)
            memoir_domain = "Cosmeto"
            return memoir_domain
        if domain == "Alimentaire":
            m.remove_layer(cluster_pharma)
            m.add_layer(cluster_alim)
            memoir_domain = "Alim"
            return memoir_domain

    if memoir_domain == "Cosmeto":
        if domain == "Tous":
            m.add_layer(cluster_pharma)
            m.add_layer(cluster_alim)
            memoir_domain = "Tous"
            return memoir_domain
        if domain == "Pharmaceutique":
            m.remove_layer(cluster_cosmeto)
            m.add_layer(cluster_pharma)
            memoir_domain = "Pharma"
            return memoir_domain
        if domain == "Alimentaire":
            m.remove_layer(cluster_cosmeto)
            m.add_layer(cluster_alim)
            memoir_domain = "Alim"
            return memoir_domain

    if memoir_domain == "Alim":
        if domain == "Tous":
            m.add_layer(cluster_pharma)
            m.add_layer(cluster_cosmeto)
            memoir_domain = "Tous"
            return memoir_domain
        if domain == "Pharmaceutique":
            m.remove_layer(cluster_alim)
            m.add_layer(cluster_pharma)
            memoir_domain = "Pharma"
            return memoir_domain
        if domain == "Cosmétique":
            m.remove_layer(cluster_alim)
            m.add_layer(cluster_cosmeto)
            memoir_domain = "Cosmeto"
            return memoir_domain

In [ ]:
drop=widgets.Dropdown(
    options=['Tous', 'Alimentaire', 'Pharmaceutique', 'Cosmétique'],

    description='Domaines :',
    value = "Tous",
    disabled=False,
)
drop.observe(drop_fonc,names='value')
# display(drop)

In [ ]:
def select_entreprises_fonc(change):
    global selection
    sel=change['new']
    sel=list(sel)
    ty=type(sel)
    print("type =",ty)
    print("selection = ",sel)
    selection = sel
    return selection

In [ ]:
List_Nom_Entrprise = []
for ligne in range(1,8):
    tupl=Donnees[ligne]
    li=list(tupl)
#     print(" ***** Ligne = ",ligne," ***********")
#     print()
#     print(li)
    result_Nom=li[2]
    List_Nom_Entrprise.append(result_Nom)
List_Nom_Entrprise.sort()

select_entreprises = widgets.SelectMultiple(
    options=List_Nom_Entrprise ,
    #value=['Oranges'],
    rows=5,
    description='Entreprises',
    disabled=False,
    layout={'width': 'max-content'}
)
select_entreprises.observe(select_entreprises_fonc,names='value')
# display(select_entreprises)

In [ ]:
def Create_pdf(_):
    global selection
    print("apres btn pdf, selection =",selection)
    infos = []
    Long_selection = len(selection)
    for select in range (Long_selection):
        for ligne in range(1,8):
            tupl=Donnees[ligne]
            li=list(tupl)
        #     print(" ***** Ligne = ",ligne," ***********")
        #     print()
        #     print(li)
            result=li[2]
            if selection[select] == result:
                
                infos.append(str(li[2]))
                infos.append(str(li[0]))
                infos.append(str(li[1]))
                infos.append(str(li[32]))
                infos.append(str(li[33]))
                infos.append(str(li[34]))
                infos.append("--> Tel : "+str(li[35]))
                infos.append("--> Mail : "+str(li[43]))
                infos.append("--> Lien : "+str(li[36]))
                infos.append("**")

                
    ###
    doc = SimpleDocTemplate("Infos_Lieux_Stages.pdf",pagesize=letter,
                            rightMargin=72,leftMargin=72,
                            topMargin=72,bottomMargin=18)
    Story=[]
    logo = "vacances.png"
#     magName = "Pythonista"
#     issueNum = 12

    formatted_time = time.ctime()

    address_parts = infos
    im = Image(logo, 2*inch, 2*inch)
    Story.append(im)
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    ptext = '%s' % formatted_time
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 12))
     
    for part in address_parts:
        ptext = '%s' % part.strip()
        if ptext == "**":
            Story.append(Spacer(1, 12))
        else:
            Story.append(Paragraph(ptext, styles["Normal"]))   
    Story.append(Spacer(1, 12))

    doc.build(Story)
#     os.startfile("Infos_Lieux_Stages.pdf")
######
#     opener = "open" if sys.platform == "darwin" else "xdg-open"
#     subprocess.call([opener, "Infos_Lieux_Stages.pdf"])
#####
#     try:
#         subprocess.Popen(['Infos_Lieux_Stages.pdf'], shell=True)
#     except IOError:
#         print ('The file is already OPENED!')
###########
#     filepath = Lieu+"\Infos_Lieux_Stages.pdf"
#     print("filepath=",filepath)
#     Yep = IFrame(filepath, width=700, height=400)
#     display(Yep)
#############
#     # url = 'https://' + 'www.google.com'
#     yop = "https://hub-binder.mybinder.ovh/user/dfialaire-test_jupyter1-0ld7sw09/files"
#     url = Lieu+"/Infos_Lieux_Stages.pdf"
#     print(url)
#     webbrowser.open(url)
#     subprocess.Popen([url], shell=True)
######
    # <a href="http://example.com/" target="_blank">link</a>

In [ ]:
Lieu = "https://hub-binder.mybinder.ovh/user/dfialaire-test_jupyter1-nwgcd7gd/files"
filepath = Lieu+"/Infos_Lieux_Stages.pdf"
print("filepath=",filepath)
Yep = IFrame(filepath, width=700, height=400)
display(Yep)

In [ ]:
yep=widgets.HTML(
    value="Hello  <a href='https://hub-binder.mybinder.ovh/user/dfialaire-test_jupyter1-nwgcd7gd/files/Infos_Lieux_Stages.pdf' target='_blank'>link</a>",
    placeholder='Some HTML',
    description='Some HTML',
)
yep


In [ ]:
#######################################
btn_pdf = Button(icon="angle-left",button_style='success')
btn_pdf.layout.width="15px"
btn_pdf.on_click( Create_pdf)
# display(btn_pdf)
#################

In [ ]:
Menus = HBox([drop,select_entreprises,btn_pdf])
display(Menus)

In [ ]:
domain = "Tous"
memoir_domain = "Tous"
selection = []

In [ ]:
markers_pharma = []
markers_cosmeto = []
markers_alim = []
#####
cent = [49.25112, 4.0271] #drop.value#
zoom = 20
m = Map(center=cent, zoom=zoom)
m.add_control(FullScreenControl())
marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))

m.add_control(SearchControl(
  position="topleft",
  url='https://nominatim.openstreetmap.org/search?format=json&q={s}',
  zoom=15,
  marker=marker
))


m.interact(zoom=(5,20,1))
#     m
#     Dessine_point(domain)
for ligne in range(1,8):
    tupl=Donnees[ligne]
    li=list(tupl)
#     print(" ***** Ligne = ",ligne," ***********")
#     print()
#     print(li)
    result=li[37]
#     print(result)
#     print(type(result))
    result=result.split(",")
    latitude=result[0]
    longitude=result[1]
    Coord = [latitude, longitude]
#     print("latitude=",latitude)
#     print("longitude=",longitude)
#     pauz=input("poz")
    infos="<u>Entreprise :</u> <b>"+str(li[2])+"</b> : <br />"+str(li[0])+" : "+str(li[1])+". <br />"+str(li[32])+", "+str(li[33])+" "+str(li[34])+"<br/> --> Tel : "+str(li[35])+"<br/> --> Mail : "+str(li[43])+"<br/> --> <a target='_blank' rel='noopener noreferrer' href='"+str(li[36])+"'>Lien </a>"
##### https://www.flaticon.com/fr/icone-premium/laboratoire-medical_3483814?related_id=3483663&origin=search    
    if li[0] == "Pharmaceutique":
        icon = AwesomeIcon(name='pills', marker_color='lightblue', icon_color='black' , icon_size=[40, 40])
#         icon = Icon(icon_url='https://img-premium.flaticon.com/png/512/3483/3483663.png?token=exp=1627657677~hmac=01ec1a6d6f237b1bdea0e5db5a905f46', icon_size=[30, 30], icon_anchor=[10,10])
    if li[0] == "Cosmétique":
        icon = AwesomeIcon(name='air-freshener', marker_color='pink', icon_color='black' , icon_size=[40, 40])
#         icon = Icon(icon_url='https://img-premium.flaticon.com/png/512/3483/3483814.png?token=exp=1627657700~hmac=a4c57b57f1129517d16ef9af1b5306ca', icon_size=[30, 30], icon_anchor=[10,10])
    if li[0] == "Alimentaire":
        icon = AwesomeIcon(name='apple-alt', marker_color='orange', icon_color='black', icon_size=[40, 40])
#         icon = Icon(icon_url='https://img-premium.flaticon.com/png/512/3483/3483835.png?token=exp=1627657167~hmac=914e0b35ab63604b16469288e1da5c19', icon_size=[30, 30], icon_anchor=[10,10])
        
    mark = Marker(location=Coord, draggable=False, icon=icon, name=li[0]) # markerColor: 'green' # marker_color
#         m.add_layer(mark)

    ###################################""
#         markers = []
#         for name, lon, lat in guj_districts_pop[["District", "Longitude", "Latitude"]].values:



    #########
    #########
    if li[0] == "Pharmaceutique":
        markers_pharma.append(mark) # Marker(location=(lat, lon), draggable=False, title=name, alt=name))
    if li[0] == "Cosmétique":
        markers_cosmeto.append(mark) # Marker(location=(lat, lon), draggable=False, title=name, alt=name))
    if li[0] == "Alimentaire":
        markers_alim.append(mark) # Marker(location=(lat, lon), draggable=False, title=name, alt=name))

        
#     cluster_pharma = MarkerCluster(markers=markers_pharma)
#     cluster_cosmeto = MarkerCluster(markers=markers_cosmeto)
#     cluster_alim = MarkerCluster(markers=markers_alim)
    
    ########
    ########
#         m.add_layer(marker_cluster)
    ##########################################

    message2 = HTML()
    message2.value = infos
    # Popup with a given location on the map:
    popup = Popup(
        location=Coord,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    ) #        child=message2,


    # Popup associated to a layer
    mark.popup = message2

#     display(m)
# m.add_layer(marker_cluster)
cluster_pharma = MarkerCluster(markers=markers_pharma)
cluster_cosmeto = MarkerCluster(markers=markers_cosmeto)
cluster_alim = MarkerCluster(markers=markers_alim)
m.add_layer(cluster_pharma)
m.add_layer(cluster_cosmeto)
m.add_layer(cluster_alim)

# control = LayersControl(position='topright')
# m.add_control(control)
m

In [ ]:
# m.save('my_map_yes.html', title='My Map')

In [ ]:
# from IPython.display import IFrame, display
# #filepath = "http://wikipedia.org" # works with websites too!
# filepath = "file.pdf"
# IFrame(filepath, width=700, height=400)

In [ ]:


res = '''Yep !'''

#FILE
filename = 'res.txt'
b64 = base64.b64encode(res.encode())
payload = b64.decode()
print(type(payload))

#BUTTONS
html_buttons = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="{filename}" href="data:text/csv;base64,{payload}" download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
</a>
</body>
</html>
'''

html_button = html_buttons.format(payload=payload,filename=filename)
display(HTML(html_button))